In [1]:
import sqlite3
import pandas as pd
from pygraphml import Graph
from pygraphml import GraphMLParser

In [2]:
conn = sqlite3.connect('rechtspraak.db')
c = conn.cursor()

In [3]:
conn.execute('''SELECT name FROM sqlite_master WHERE type='table';''').fetchall()

[('uitspraken_meta',),
 ('uitspraken',),
 ('uitspraken_links',),
 ('uitspraken_articles',),
 ('temp_ids',)]

In [4]:
#Get the links to other HR cases
links = pd.DataFrame(c.execute('''select l.id, l.reference
            from uitspraken_links l join 
                ( select id from uitspraken_meta ) s
                on s.id = l.reference
        ''').fetchall(), columns=['id', 'reference'])

In [5]:


def create_temp_ids(ids, c):
    c.execute('''
            DROP TABLE IF EXISTS temp_ids
    ''')
    c.execute(''' CREATE TABLE temp_ids
                (id text)''')
    for id0 in ids:
        c.execute(''' INSERT INTO temp_ids VALUES (?) '''
                  , (id0,))

def get_descriptions(tablename):
    descriptions = pd.DataFrame(c.execute('''select l.id, r.summary
            from ( 
                select distinct id from {}
                ) l 
            left join uitspraken_meta r
                on l.id = r.id
        '''.format(tablename)).fetchall(), columns=['id', 'description'])
    descriptions = descriptions.set_index('id')
    return descriptions

def get_articles(tablename):
    articles = pd.DataFrame(c.execute('''select l.id, r.article_name, r.article_number
            from ( 
                select distinct id from {}
                ) l 
            left join uitspraken_articles r
                on l.id = r.id
            order by r.cnt desc
        '''.format(tablename)).fetchall(), columns=['id', 'article_name', 'article_number'])
    articles['article'] =  articles['article_number']+' '+articles['article_name']
    articles_grouped = articles.groupby('id')['article'].apply(lambda l: [x for x in l if not pd.isnull(x)])
    return articles_grouped

def get_links_and_meta(ids, c):
    create_temp_ids(ids, c)
    descriptions = get_descriptions('temp_ids')
    articles = get_articles('temp_ids')
    descr_articles = descriptions.merge(pd.DataFrame(articles), left_index=True, right_index=True)
    return descr_articles

In [8]:
ids = list(set(pd.concat((links.id, links.reference))))
descr_articles = get_links_and_meta(ids, c)

In [9]:

def year_from_id(text):
    return text.split(':')[3]


def make_graph(descriptions, links):
    g = Graph()
    node_dict = {}
    for ecli in descriptions.index:
        n = g.add_node(ecli)
        n['description'] = descriptions['description'][ecli]
        n['articles'] = descriptions['article'][ecli]
        n['year'] = int(year_from_id(ecli))
        node_dict[ecli] = n
    # Add edges
    for i in links.index:
        n1, n2 = (links['id'][i], links['reference'][i])
        g.add_edge(node_dict[n1], node_dict[n2], directed=True)
    return g

def save_graph(graph, outputpath):
    parser = GraphMLParser()
    parser.write(graph, outputpath)

In [10]:
outputpath = '/media/sf_VBox_Shared/CaseLaw/graphs/eclilinks.graphml'
g = make_graph(descr_articles, links)
save_graph(g, outputpath)

In [12]:
#Smaller graph: search for werkgever:
lower = descr_articles['description'].str.lower()
descriptions_small = descr_articles[lower.str.contains('aansprakelijkheid')]

In [13]:
links_small = links.merge(descriptions_small, left_on='id', right_index=True)[['id', 'reference']]

In [14]:
ids_small = pd.concat([links_small['id'], links_small['reference']]).unique()

In [16]:
descr_articles_small = get_links_and_meta(ids_small, c)
descr_articles_small.shape

(112, 2)

In [17]:
outputpath = '/media/sf_VBox_Shared/CaseLaw/graphs/eclilinks_small.graphml'
g2 = make_graph(descr_articles_small, links_small)
save_graph(g2, outputpath)

## existing network from Gijs

In [18]:
filepath = '/media/sf_VBox_Shared/CaseLaw/graphs/test3 wg aanspr.csv'

In [19]:
links_wgaans = pd.read_csv(filepath, sep=';', header=0)

In [20]:
links_wgaans.columns = ['id', 'reference']

In [21]:
links_wgaans = links_wgaans.drop_duplicates()

In [22]:
ids_wgaans = list(pd.concat([links_wgaans['id'], links_wgaans['reference']]).unique())

In [23]:
conn = sqlite3.connect('rechtspraak.db')
c = conn.cursor()

In [24]:
descriptions_wgaans = get_descriptions('temp_ids')
articles_wgaans = get_articles('temp_ids')
print(descriptions_wgaans.shape)
print(articles_wgaans.shape)

(0, 1)
(0,)


In [25]:
descr_articles_wgaans = get_links_and_meta(ids_wgaans, c)
descr_articles_wgaans.shape

(156, 2)

In [26]:
outputpath = '/media/sf_VBox_Shared/CaseLaw/graphs/eclilinks_wgaans.graphml'
g3 = make_graph(descr_articles_wgaans, links_wgaans)
save_graph(g3, outputpath)

In [27]:
print(len(ids_wgaans))
descr_articles_wgaans.shape

156


(156, 2)

In [28]:
c.execute('''select count(*) from temp_ids
        ''').fetchall()

[(156,)]

In [29]:
conn.close()

In [30]:
descr_articles_wgaans

,description,article
id,,
ECLI:NL:HR:1969:AC4973,None,[]
ECLI:NL:HR:1975:AC5607,None,[]
ECLI:NL:HR:1976:AC5837,None,[]
ECLI:NL:HR:1978:AC3514,None,[]
ECLI:NL:HR:1982:AC0442,None,[]
ECLI:NL:HR:1982:AG4420,None,[]
ECLI:NL:HR:1983:AG4611,None,[]
ECLI:NL:HR:1987:AC3826,None,[]
ECLI:NL:HR:1987:AG5504,None,[]
